In [ ]:
# Import package

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import time
from time import sleep
import pandas as pd
import numpy as np

print('- Finish importing package')

In [ ]:
# Open Chrome

DRIVER_PATH = '/Users/humi.mochi/Downloads/chromedriver'
driver = webdriver.Chrome(executable_path = DRIVER_PATH)
print('- Finish opening Chrome')
sleep(3)

In [ ]:
# # Import email and password
# credential = open("login_topcv.txt") #Create a file .txt contain private information Login
# line = credential.readlines()
# email = line[0]
# password = line[1]
# print('- Finish importing the login credentials')

# # Key in email
# email_field = driver.find_element(By.XPATH,'Copy xpath')
# email_field.send_keys(email)
# print('- Finish keying in email')
# sleep(3)

# #Key in password
# password_field = driver.find_element(By.XPATH,'Copy xpath')
# password_field.send_keys(password)
# print('_ Finish keying in password')
# sleep(3)

# # Click login button
# login_field = driver.find_element(By.XPATH,'Copy xpath')
# login_field.click()
# print('- Finish logging in')
# sleep(3)

In [ ]:
# Login Indeed login site

url = 'https://www.indeed.com/stc?_ga=2.40830050.1924450139.1656265669-1082206623.1656265669'
driver.get(url)
print('- Finish getting url')
sleep(3)

In [ ]:
#Record time run crawl data

starttime = time.time()

In [ ]:
# Input and execute keyword searching

def search_keyword(word):
    '''
        Func uses for input and execute keyword searching
        Params: a specific word
    '''
    search_job = driver.find_element(By.XPATH, '//*[@id="text-input-what"]')
    search_job.send_keys(word)
    search_button = driver.find_element(By.XPATH, '//*[@id="jobsearch"]/button')
    search_button.click()

In [ ]:
# Navigate to the next page

def goto_nextpage(word, page):
    '''
        Func uses to navigate to the next page
        Params: a keyword, page number
    '''
    if ' ' in word:
        uri = word.split(' ')
        word1 = uri[0]
        word2 = uri[1]
        try:
            next_page = driver.get(f'https://www.indeed.com/jobs?q={word1}%20{word2}&start={(page)*10}')

        except:
            next_page = driver.get(f'https://www.indeed.com/jobs?q={word1}%20{word2}&start={(page)*10}')
    else:
        try:
            next_page = driver.get(f'https://www.indeed.com/jobs?q={word}&start={(page)*10}')

        except:
            next_page = driver.get(f'https://www.indeed.com/jobs?q={word}&start={(page)*10}')

In [ ]:
# Initialize count variable to order csv files
count = 1

# Input a list of keywords
# keywords = input_keywords()
keywords = ['data science', 'development', 'data analyst', 'data engineer']
for keyword in keywords:

    titles = []
    companies = []
    locations =[]
    salaries = []
    types = []
    categories = []

    # Then print the new keyword to keep track
    print(keyword)
    
    # Execute searching keyword and return the first result page
    search_keyword(keyword)
    
    # First page
    page = 1

    # Lopp from page 1 to page 65
    for page in range(1, 66):
        # Print page number to console
        print(f'{keyword} - Page {page}')
        
        # Find the list of job cards
        job_card = driver.find_elements(By.XPATH, '//div[contains(@class,"job_seen_beacon")]')
        
        # Iterate to job cards
        for job in job_card:
            # Find DOM element that contains job's tilte
            try:
                title  = job.find_element(By.XPATH,'//*[@class="jobTitle"]/a/span').text
            except:
                # If cannot find the element, set title is NULL
                title  = None
                # Append the title text to list
            titles.append(title)

            # The belows are same as finding title
            try:    
                company  = job.find_element(By.XPATH,'.//span[@class="companyName"]').text
            except:
                company  = None
            companies.append(company)

            try:
                location  = job.find_element(By.XPATH,'.//div[@class="companyLocation"]').text
            except:
                location  = None
            locations.append(location)

            try:
                salary = job.find_element(By.XPATH,'.//span[@class="estimated-salary"]').text
            except:
                salary = None
            salaries.append(salary)

            try:
                type = job.find_element(By.XPATH,'.//div[@class="metadata"]/div[@class="attribute_snippet"]').text
            except:
                type = None
            types.append(type)
            categories.append(keyword)

            # print(f'{title}; {company}; {location}; {salary}; {type}')
            
        # Navigate to the next page
        goto_nextpage(keyword, page)

        
    # Mark a keyword crawling process is done.
    print('crawling completed.')

    driver.get('https://www.indeed.com/stc?_ga=2.63393007.1966857820.1656152873-1453494806.1656152873')

    # Start writing csv file process.
    print('Starting wrtie to csv file....')
    df_da=pd.DataFrame()
    df_da['Title']=titles
    df_da['Company']=companies
    df_da['Location']=locations
    df_da['Salary']=salaries
    df_da['Type']=types
    df_da['Category']=categories

    df_da.to_csv(f'output\\part00{count}.csv', index=False)

    # Increase count for next file
    count += 1

    # Fnish writing csv file process.
    print(f'Finish write {keyword} data to csv file.')

In [ ]:
endtime = time.time()
process_time = round((endtime-starttime)/60,2)
print("Processing Time: {} min".format(process_time))

In [ ]:
# all_files = glob.glob("output/*.csv")
# # print(all_files)
# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0, delimiter=';')
#     li.append(df)

# df = pd.concat(li, axis=0, ignore_index=True)
# print(frame)
# df = df[['Title', 'Company', 'Location', 'Salary', 'Type', 'Category']].drop_duplicates()
# print(df)